# Linear model

## Import packages

In [ ]:
import os
import random
import glob
import csv
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

## Parameters

In [ ]:
TRA_PATH = './data/full_reduced_train.csv'
TST_PATH = './data/not_full_reduced_train.csv'
LABEL_PATH = './data/full_record.csv'
DEVICE_ID = 2
SEED = 5566
NUM_ECPOCH = 3
transform = None

## device config

In [ ]:
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")

## functions

In [ ]:
def load_train_data(train_path, label_path, valid_ratio=0.12):
    train_label_original = pd.read_csv(label_path)['top1'].values.tolist()
    category = [2, 6, 10, 12, 13, 15, 18, 19, 21, 22, 25, 26, 36, 37, 39, 48]
    train_label = np.zeros(len(train_label_original))
    for i in range(len(train_label_original)):
        for j in range(len(category)):
            if(train_label_original[i] == category[j]):
                train_label[i] = j
    df = pd.read_csv(train_path)
    train_predict = df[["masts","educd","trdtp","naty","poscd","cuorg","slam","gender_code","age"]].to_numpy()
    mean = np.mean(train_predict, axis=0)
    std = np.std(train_predict, axis=0)
    for i in range(len(train_predict)):
        train_predict[i] = (train_predict[i] - mean) / std
    train_data = list(zip(train_predict, train_label))
    random.shuffle(train_data)
    
    split_len = int(len(train_data) * valid_ratio)
    train_set = train_data[split_len:]
    valid_set = train_data[:split_len]
    
    return train_set, valid_set

def load_test_data(test_path):
    df = pd.read_csv(test_path)
    test_set = df[["masts","educd","trdtp","naty","poscd","cuorg","slam","gender_code","age"]].to_numpy()
    mean = np.mean(test_set, axis=0)
    std = np.std(test_set, axis=0)
    for i in range(len(test_set)):
        test_set[i] = (test_set[i] - mean) / std
    return test_set

train_set, valid_set = load_train_data(TRA_PATH, LABEL_PATH)
test_set = load_test_data(TST_PATH)

## Dataset

In [ ]:
class FaceExpressionDataset(Dataset):
    def __init__(self, data, augment=None):
        self.data = data
        self.augment = augment

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        label = self.data[idx][1]
        return self.data[idx][0], label

In [ ]:
class TestingDataset(Dataset):
    def __init__(self, data, augment=None):
        self.data = data
        self.augment = augment      

    def __len__(self):
        return len(self.data)
      
    def __getitem__(self, idx):
        return self.data[idx]

## Input train set and data set

In [ ]:
train_dataset = FaceExpressionDataset(train_set, transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

valid_dataset = FaceExpressionDataset(valid_set, transform)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)

test_dataset = TestingDataset(test_set,transform)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

## Neural network

In [ ]:
class FaceExpressionNet(nn.Module):
    def __init__(self):
        super(FaceExpressionNet, self).__init__()
        # TODO    
        self.fc = nn.Sequential(
            nn.Linear(9, 64),
            nn.ReLU(),
            nn.Linear(64,64),
            nn.ReLU(),
            torch.nn.Dropout(0.5),
            nn.Linear(64, 16)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

## Training function process

In [ ]:
def train(train_loader, model, loss_fn, use_gpu=True):
    model.train()
    train_loss = []
    train_acc = []
    for (data, label) in train_loader:
        if use_gpu:
            data = data.to(device)
            label = label.to(device)
        optimizer.zero_grad()
        data = torch.tensor(data, dtype=torch.float32)
        label= torch.tensor(label, dtype=torch.long)
        output = model(data)
        loss = loss_fn(output, label)
        loss.backward()            
        optimizer.step()
        with torch.no_grad():
            predict = torch.argmax(output, dim=-1)
            acc = np.mean((label == predict).cpu().numpy())
            train_acc.append(acc)
            train_loss.append(loss.item())
    print("Epoch: {}, train Loss: {:.4f}, train Acc: {:.4f}".format(epoch + 1, np.mean(train_loss), np.mean(train_acc)))
    
def valid(valid_loader, model, loss_fn, use_gpu=True):
    model.eval()
    with torch.no_grad():
        valid_loss = []
        valid_acc = []
        for idx, (data, label) in enumerate(valid_loader):
            if use_gpu:
                data = data.to(device)
                label = label.to(device)
            data = torch.tensor(data, dtype=torch.float32)
            label= torch.tensor(label, dtype=torch.long)
            
            output = model(data)
            loss = loss_fn(output, label)
            
            
            predict = torch.argmax(output, dim=-1)
            acc = (label == predict).cpu().tolist()
            valid_loss.append(loss.item())
            valid_acc += acc
       
        valid_acc = np.mean(valid_acc)
        valid_loss = np.mean(valid_loss)
        print("Epoch: {}, valid Loss: {:.4f}, valid Acc: {:.4f}".format(epoch + 1, valid_loss, valid_acc))
    return valid_acc

def save_checkpoint(valid_acc, acc_record, epoch, prefix='model'):
#     you can define the condition to save model :)
    if valid_acc >= np.mean(acc_record[-5:]):    
        checkpoint_path = f'{prefix}.pth'
        torch.save(model.state_dict(), checkpoint_path)
        print('model saved to %s' % checkpoint_path)

def early_stop(valid_acc):
    # TODO
    return False

## Main Function

In [ ]:
model = FaceExpressionNet()
if use_gpu:
    model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

acc_record = []
for epoch in range(NUM_ECPOCH):
    train(train_loader, model, loss_fn, use_gpu)
    valid_acc = valid(valid_loader, model, loss_fn, use_gpu=True)
    acc_record.append(valid_acc)

    save_checkpoint(valid_acc, acc_record, epoch, prefix='model')
    if early_stop(valid_acc):
        break

    print('########################################################')

## Testing

In [ ]:
def test(test_loader, model, file_name='./data/predict_not_full.csv'):
    with torch.no_grad():
        predict_result = np.zeros((0,3))
        for idx, data in enumerate(test_loader):
            if use_gpu:
                data = data.to(device)
            data = torch.tensor(data, dtype=torch.float32)
            output = model(data)
            output_sort = np.argsort(output)
            output_top3 = output_sort[:,(-1,-2,-3)].numpy()
            predict_result = np.concatenate((predict_result, output_top3), axis=0)

    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['chid', 'top1','top2','top3'])
        test_df = pd.read_csv(TST_PATH)
        category = [2, 6, 10, 12, 13, 15, 18, 19, 21, 22, 25, 26, 36, 37, 39, 48]
        for i in range(len(predict_result)):
            writer.writerow([test_df['chid'][i], str(category[int(predict_result[i][0])]),
                             str(category[int(predict_result[i][1])]), str(category[int(predict_result[i][2])])])

test(test_loader, model)